# Motor Control Testing

Test notebook for the **DigitalTwinMotorDriver** and **Motor** device class.

Uses the new telescope-mcp architecture - no hardware required!

---

## Architecture Overview

```
Motor (device)          ← High-level async API
    ↓
MotorInstance           ← Runtime instance (position, status)
    ↓
MotorDriver             ← Factory for instances (DigitalTwinMotorDriver)
```

## Motor Configuration

| Axis | Range | Home | Steps/Degree |
|------|-------|------|--------------|
| ALT | 0° (zenith) → ~90° (horizon) | 0 (zenith) | ~19,167 |
| AZ | ±110,000 steps (~±50°) | 0 (center) | ~2,222 |

---
## 1. Configuration

Toggle between **Digital Twin** (simulation) and **Hardware** (real motors) mode.

In [52]:
# ============================================
# CONFIGURATION - Choose hardware or simulation
# ============================================

# Set to True for real hardware, False for digital twin simulation
#USE_HARDWARE = False
USE_HARDWARE = True

# Hardware settings (only used when USE_HARDWARE=True)
MOTOR_BAUD_RATE = 115200

# Known Teensy hardware ID (from serial port detection)
# Format: "USB VID:PID=16C0:0483 SER=<serial_number> LOCATION=<bus-port>"
# Run the port detection cell to find your Teensy's hwid, then paste it here
TEENSY_HWID = None  # Set to string like "USB VID:PID=16C0:0483 SER=12345678 LOCATION=1-2"

print(f"✓ Mode: {'HARDWARE' if USE_HARDWARE else 'DIGITAL TWIN'}")
if TEENSY_HWID:
    print(f"✓ Teensy hwid: {TEENSY_HWID}")

✓ Mode: HARDWARE


In [53]:
# ============================================
# DETECT SERIAL PORTS
# Shows available ports and identifies Teensy
# Only runs detection in hardware mode
# ============================================

import serial.tools.list_ports

teensy_port = None

if USE_HARDWARE:
    print("Scanning serial ports...\n")
    ports = list(serial.tools.list_ports.comports())

    # Look for Teensy (Teensyduino shows as "USB Serial" or contains "Teensy")
    for port in ports:
        desc = (port.description or "").lower()
        mfr = (port.manufacturer or "").lower()
        
        # Teensy typically shows as "USB Serial" with Teensyduino or PJRC
        is_teensy = (
            "teensy" in desc or 
            "teensy" in mfr or
            "pjrc" in mfr or
            (port.vid == 0x16C0 and port.pid == 0x0483)  # Teensy USB VID:PID
        )
        
        marker = " ← TEENSY" if is_teensy else ""
        print(f"  {port.device}{marker}")
        print(f"    Description: {port.description}")
        print(f"    Manufacturer: {port.manufacturer}")
        print(f"    VID:PID: {hex(port.vid)}:{hex(port.pid)}" if port.vid else "    VID:PID: N/A")
        print()
        
        if is_teensy and teensy_port is None:
            teensy_port = port.device

    print("─" * 50)
    if teensy_port:
        print(f"✓ Teensy detected: {teensy_port}")
    else:
        print("❌ No Teensy detected - hardware mode will fail!")
        print("   Connect Teensy and re-run this cell.")
else:
    print("ℹ️ Digital twin mode - skipping serial port detection")

Scanning serial ports...

  /dev/ttyS31
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS30
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS29
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS28
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS27
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS26
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS25
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS24
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS23
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS22
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS21
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS20
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS19
    Description: n/a
    Manufacturer: None
    

---
## 2. Setup

Import the motor architecture from telescope-mcp.

In [54]:
# ============================================
# SETUP - Import motor architecture
# Run this after kernel restart to pick up code changes
# ============================================

# Force reimport of modules (useful during development)
# Order matters! Reload base types first, then implementations
import importlib
import telescope_mcp.drivers.motors.types
import telescope_mcp.drivers.motors.twin
import telescope_mcp.drivers.motors.serial_controller  # Hardware driver
import telescope_mcp.drivers.motors  # Also reload the __init__.py

importlib.reload(telescope_mcp.drivers.motors.types)
importlib.reload(telescope_mcp.drivers.motors.twin)
importlib.reload(telescope_mcp.drivers.motors.serial_controller)
importlib.reload(telescope_mcp.drivers.motors)

from telescope_mcp.drivers.motors import (
    DigitalTwinMotorDriver,
    DigitalTwinMotorConfig,
    DigitalTwinMotorInstance,
    SerialMotorDriver,  # Real hardware driver
    MotorStatus,
    MotorInfo,
    MotorType,
)
from telescope_mcp.devices import Motor, MotorConfig
import asyncio

print("✓ Motor architecture imported")

✓ Motor architecture imported


In [55]:
# ============================================
# CREATE MOTOR DRIVER
# Uses hardware or digital twin based on USE_HARDWARE setting
# ============================================

if USE_HARDWARE:
    # Real hardware - connect to Teensy motor controller
    if not teensy_port:
        raise RuntimeError(
            "No Teensy detected! Connect the motor controller and re-run the port detection cell."
        )
    
    serial_port = teensy_port
    driver = SerialMotorDriver(baudrate=MOTOR_BAUD_RATE)
    print(f"✓ SerialMotorDriver created")
    print(f"  Serial port: {serial_port}")
    print(f"  Baud rate: {MOTOR_BAUD_RATE}")
else:
    # Digital twin - simulated motors
    serial_port = None  # Not used in digital twin mode
    config = DigitalTwinMotorConfig(
        # Position limits (matching real hardware)
        altitude_min_steps=0,       # Zenith (straight up)
        altitude_max_steps=140000,  # Horizon
        azimuth_min_steps=-110000,  # CCW limit
        azimuth_max_steps=110000,   # CW limit
        
        # Home positions
        altitude_home_steps=0,      # Home at zenith
        azimuth_home_steps=0,       # Home at center
        
        # Timing simulation
        slew_speed_steps_per_sec=5000.0,
        acceleration_time_sec=0.5,
        simulate_timing=True,
    )
    driver = DigitalTwinMotorDriver(config)
    print(f"✓ DigitalTwinMotorDriver created")
    print(f"  Config: {config}")

✓ SerialMotorDriver created
  Serial port: /dev/ttyACM2
  Baud rate: 115200


In [56]:
# ============================================
# STEPPER_AMIS RAW CONNECTION
# Direct connection bypassing telescope-mcp for diagnostics
# ============================================
import sys
sys.path.insert(0, '/home/mark/src/at_stepper_amis')

from stepper_amis import Stepper_amis, getAllHwids
from pprint import pprint

# Show available serial devices
print("Available serial devices:")
pprint(getAllHwids())

if USE_HARDWARE and teensy_port:
    # Close telescope-mcp connection first to avoid port conflict
    try:
        driver.close()
        print("\n✓ Closed telescope-mcp driver to release serial port")
    except:
        pass
    
    # Use known hwid if configured, otherwise search for it
    if TEENSY_HWID:
        teensy_hwid = TEENSY_HWID
        print(f"\nUsing configured Teensy hwid: {teensy_hwid}")
    else:
        # Find hwid for the Teensy port (fallback)
        from serial.tools.list_ports import comports
        teensy_hwid = None
        for cp in comports():
            if str(cp).split()[0] == teensy_port:
                teensy_hwid = cp.hwid
                break
        
        if teensy_hwid:
            print(f"\nTeensy hwid (detected): {teensy_hwid}")
            print(f"💡 TIP: Add this to TEENSY_HWID in config cell for faster connection")
        else:
            print(f"❌ Could not find hwid for {teensy_port}")
    
    if teensy_hwid:
        # Connect using hwid (Stepper_amis API)
        sa = Stepper_amis(teensy_hwid)
        print(f"✓ Stepper_amis connected via {teensy_port}")
    else:
        sa = None
else:
    print("❌ Hardware mode not enabled or no Teensy port detected")
    sa = None

Available serial devices:
{'/dev/ttyACM0': {'hwid': 'USB VID:PID=2341:805A SER=D6887F2418ABF612 '
                          'LOCATION=1-2:1.0'},
 '/dev/ttyACM2': {'hwid': 'USB VID:PID=16C0:0483 SER=4516910 '
                          'LOCATION=1-3.4:1.0'},
 '/dev/ttyS0': {'hwid': 'n/a'},
 '/dev/ttyS1': {'hwid': 'n/a'},
 '/dev/ttyS10': {'hwid': 'n/a'},
 '/dev/ttyS11': {'hwid': 'n/a'},
 '/dev/ttyS12': {'hwid': 'n/a'},
 '/dev/ttyS13': {'hwid': 'n/a'},
 '/dev/ttyS14': {'hwid': 'n/a'},
 '/dev/ttyS15': {'hwid': 'n/a'},
 '/dev/ttyS16': {'hwid': 'n/a'},
 '/dev/ttyS17': {'hwid': 'n/a'},
 '/dev/ttyS18': {'hwid': 'n/a'},
 '/dev/ttyS19': {'hwid': 'n/a'},
 '/dev/ttyS2': {'hwid': 'n/a'},
 '/dev/ttyS20': {'hwid': 'n/a'},
 '/dev/ttyS21': {'hwid': 'n/a'},
 '/dev/ttyS22': {'hwid': 'n/a'},
 '/dev/ttyS23': {'hwid': 'n/a'},
 '/dev/ttyS24': {'hwid': 'n/a'},
 '/dev/ttyS25': {'hwid': 'n/a'},
 '/dev/ttyS26': {'hwid': 'n/a'},
 '/dev/ttyS27': {'hwid': 'n/a'},
 '/dev/ttyS28': {'hwid': 'n/a'},
 '/dev/ttyS29': {'hw

In [51]:
# ============================================
# TEST MOVE WITH STEPPER_AMIS DIRECTLY
# Bypasses telescope-mcp to test raw motor movement
# ============================================

TEST_AXIS = 1 # ALT

TEST_MICROSTEPS = -10000 # positive is clockwise

if sa is None:
    print("❌ No connection - run the connection cell first")
else:
    print(f"Testing Axis {TEST_AXIS} with {TEST_MICROSTEPS} microsteps...")

    # Get position before
    status_before = sa.status(TEST_AXIS)
    pos_before = status_before.get('x', 0)
    print(f"  Position before: {pos_before}")

    # Get current settings for this move
    vel = status_before.get('speed', 0)
    accel = status_before.get('accel', 0)
    current = status_before.get('iRun', 0)
    print(f"  Velocity: {vel} μsteps/sec")
    print(f"  Acceleration: {accel} sec")
    print(f"  Current: {current} mA")

    # Do the move (move2 is relative move)
    print(f"\n  Moving +{TEST_MICROSTEPS} microsteps...")
    sa.move2(TEST_AXIS, TEST_MICROSTEPS)

    # Check new position
    status_after = sa.status(TEST_AXIS)


Testing Axis 1 with -10000 microsteps...
  Position before: 0
  Velocity: 1100.0 μsteps/sec
  Acceleration: 0.1 sec
  Current: 0 mA

  Moving +-10000 microsteps...


In [57]:
# ============================================
# CLOSE STEPPER_AMIS CONNECTION
# Run before going back to telescope-mcp driver
# ============================================

try:
    sa.close()
    print("✓ Stepper_amis serial connection closed")
    print("  You can now re-run the telescope-mcp driver cells")
except Exception as e:
    print(f"Connection close error: {e}")
    print("  You can now re-run the telescope-mcp driver cells")
except:
    print("Connection already closed or not opened")

✓ Stepper_amis serial connection closed
  You can now re-run the telescope-mcp driver cells
